In [63]:
import requests
from bs4 import BeautifulSoup
import concurrent.futures
import time, random

import json
import re
import os

job_links = []

# Header giả lập trình duyệt thật
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/119.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    # 👇 Tắt yêu cầu nén gzip/br
    "Accept-Encoding": "identity",
    "Accept-Language": "vi-VN,vi;q=0.9,en;q=0.8",
    "Connection": "keep-alive",
}

def crawl_page(page):
    url = f"https://www.topcv.vn/viec-lam-it?page={page}"
    try:
        # Delay ngẫu nhiên tránh bị chặn
        time.sleep(random.uniform(2, 12))

        response = requests.get(url, headers=headers, timeout=15)
        response.encoding = response.apparent_encoding  # tự đoán encoding

        # Lưu HTML của trang ra file để kiểm tra
        #with open(f"page_{page}.html", "wb") as f:
            #f.write(response.content)

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")
            all_job = soup.find_all("div", class_="job-item-2")
            links = []
            for job in all_job:
                a_tag = job.find("a", href=True)
                if a_tag:
                    links.append(a_tag["href"])
            print(f"Trang {page}: {len(links)} links")
            return links
        else:
            print(f"Trang {page} lỗi HTTP {response.status_code}")
    except Exception as e:
        print(f"Lỗi trang {page}: {e}")
    return []


# === Chạy đa luồng nhẹ (đừng quá cao nếu không bị chặn) ===
with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
    results = executor.map(crawl_page, range(2, 47))  # thử trước 20 trang
    for r in results:
        job_links.extend(r)

print("\nTổng số link thu được:", len(job_links))

Trang 2: 50 links
Trang 4: 50 links
Trang 3: 50 links
Trang 6: 50 links
Trang 5: 50 links
Trang 8: 50 links
Lỗi trang 7: HTTPSConnectionPool(host='www.topcv.vn', port=443): Read timed out. (read timeout=15)
Trang 10: 50 links
Trang 9: 50 links
Trang 13: 50 links
Trang 11: 50 links
Trang 12: 50 links
Trang 15: 50 links
Trang 17: 50 links
Trang 14: 50 links
Trang 16: 50 links
Trang 18: 50 links
Trang 19: 50 links
Trang 20: 50 links
Trang 21: 50 links
Trang 22: 50 links
Trang 24: 50 links
Trang 23: 50 links
Trang 25: 50 links
Trang 26: 50 links
Trang 27: 50 links
Trang 28: 50 links
Trang 29: 50 links
Trang 30: 50 links
Trang 32: 50 links
Trang 31: 50 links
Trang 33: 50 links
Trang 34: 50 links
Trang 35: 50 links
Lỗi trang 36: HTTPSConnectionPool(host='www.topcv.vn', port=443): Read timed out. (read timeout=15)
Trang 37: 50 links
Trang 39: 50 links
Trang 38: 50 links
Lỗi trang 42: HTTPSConnectionPool(host='www.topcv.vn', port=443): Read timed out. (read timeout=15)
Lỗi trang 41: HTTPSConne

In [57]:
job_links[0]

'https://www.topcv.vn/viec-lam/ky-thuat-it-phong-bao-hanh-nam-thu-nhap-upto-11-trieu-tai-ha-noi/1930097.html?ta_source=ITJobs_LinkDetail'

In [58]:
def link_to_json(url, json_file_name):
    try:
        # Lấy soup
        time.sleep(random.uniform(2, 12))
        response = requests.get(url, headers=headers, timeout=15)

        if response.status_code == 429:
            time.sleep(10)
            response = requests.get(url, headers=headers, timeout=15)
            
        if response.status_code != 200:
            print(f"Lỗi khi truy cập {url} - Mã {response.status_code}")
            return None

        soup = BeautifulSoup(response.text, "html.parser")

        # Lấy job_description_content
        job_description_content = soup.find("div", class_=lambda c: c and "job-description" in c)
        if not job_description_content:
            print(f"⚠️ Không tìm thấy phần job-description trong {url}")
            return None

        job_text = job_description_content.get_text(separator="\n", strip=True)

        output_lines = []
        # === Tên công ty ===
        company_name_label = soup.find("div", class_="company-name-label")
        if company_name_label:
            a_tag = company_name_label.find("a")
            if a_tag:
                company_name = a_tag.get_text(strip=True)
                output_lines.append("=== Tên công ty ===")
                output_lines.append(company_name)

        # === Các phần thông tin khác ===
        job_detail_info = soup.find_all("div", class_="job-detail__info--section")
        for section in job_detail_info:
            title_tag = section.find("div", class_="job-detail__info--section-content-title")
            value_tag = section.find("div", class_="job-detail__info--section-content-value")

            if value_tag:
                a_tag = value_tag.find("a")
                if a_tag:
                    value_text = a_tag.get_text(strip=True)
                else:
                    value_text = value_tag.get_text(strip=True)
            else:
                value_text = ""

            title_text = title_tag.get_text(strip=True) if title_tag else "Không có tiêu đề"

            output_lines.append(f"=== {title_text} ===")
            output_lines.append(value_text)

        # === Các phần mô tả công việc ===
        items = job_description_content.find_all("div", class_="job-description__item")
        for item in items:
            title = item.find("h3").get_text(strip=True)
            content = item.find("div", class_="job-description__item--content").get_text(separator="\n", strip=True)
            output_lines.append(f"=== {title} ===")
            output_lines.append(content)

        # Ghép lại toàn bộ text
        text = "\n".join(output_lines)

        # Tách dữ liệu theo pattern
        pattern = r"===\s*(.*?)\s*===\s*([\s\S]*?)(?===|$)"
        matches = re.findall(pattern, text)

        job_info = {}
        for title, content in matches:
            clean_title = title.strip()
            clean_content = content.strip()
            job_info[clean_title] = clean_content

        # Đọc file hiện có
        filename = json_file_name
        if os.path.exists(filename):
            with open(filename, "r", encoding="utf-8") as f:
                try:
                    data = json.load(f)
                except json.JSONDecodeError:
                    data = []
        else:
            data = []

        # Ghi thêm bản ghi mới
        data.append(job_info)
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(data, f, ensure_ascii=False, indent=2)

        print(f"✅ Đã thêm một bản ghi vào {filename}")
        return job_info

    except Exception as e:
        print(f"❌ Lỗi khi xử lý {url}: {e}")
        return None

In [59]:
url = job_links[0]
print(url)

https://www.topcv.vn/viec-lam/ky-thuat-it-phong-bao-hanh-nam-thu-nhap-upto-11-trieu-tai-ha-noi/1930097.html?ta_source=ITJobs_LinkDetail


In [60]:
link_to_json(url,"jobs.json")

✅ Đã thêm một bản ghi vào jobs.json


{'Tên công ty': 'CÔNG TY TNHH ĐIỆN TỬ CÔNG NGHỆ TƯỜNG AN – TAKO',
 'Mức lương': '9 - 11 triệu',
 'Địa điểm': 'Hà Nội',
 'Kinh nghiệm': '1 năm',
 'Mô tả công việc': 'Công ty là NPP lâu năm (25 năm) của hãng các nhãn hàng lớn trên thị trường như TP-Link, AULA, Firstplayer cần tuyển NV làm việc tại phòng kỹ thuật bảo hành của công ty bao gồm các CV sau:\nPhụ trách test lỗi các sản phẩm thiết bị mạng và thiết bị ngoại vi như bàn phím, chuột, nguồn, case,....\nKiểm tra, cài đặt các thiết bị mạng như router, switch mạng, wifi,...\nHướng dẫn, hỗ trợ kỹ thuật về hướng dẫn sử dụng, kiểm tra lỗi của thiết bị mạng và thiết bị ngoại vi cho đại lý và khách hàng qua mail, điện thoại, zalo,...\nLàm việc bộ phận kỹ thuật, bảo hành của nhà cung cấp của công ty.\nHỗ trợ 1 phần kỹ thuật điện tử sửa các sản phẩm điện tử (nếu chưa có kinh nghiệm sẽ được đào tạo)\nThời gian làm việc: từ 7h45-12h, 13h30-17h30 thứ 2 đến thứ 7, CN và ngày lễ nghỉ',
 'Yêu cầu ứng viên': 'Nam, tốt nghiệp cao đẳng, đại học chuyên

In [65]:
for i in range(1, 2001):
    # Tính số thứ tự file (chia 100, làm tròn lên)
    number_of_file = (i - 1) // 100 + 1

    # Tạo tên file tương ứng
    file_name = f"job_data_{number_of_file}.json"

    # Gọi hàm lưu dữ liệu
    link_to_json(job_links[i], file_name)

    print(f"Đã lưu job {i} vào file {file_name}")

    


✅ Đã thêm một bản ghi vào job_data_1.json
Đã lưu job 1 vào file job_data_1.json
✅ Đã thêm một bản ghi vào job_data_1.json
Đã lưu job 2 vào file job_data_1.json
✅ Đã thêm một bản ghi vào job_data_1.json
Đã lưu job 3 vào file job_data_1.json
✅ Đã thêm một bản ghi vào job_data_1.json
Đã lưu job 4 vào file job_data_1.json
✅ Đã thêm một bản ghi vào job_data_1.json
Đã lưu job 5 vào file job_data_1.json
✅ Đã thêm một bản ghi vào job_data_1.json
Đã lưu job 6 vào file job_data_1.json
✅ Đã thêm một bản ghi vào job_data_1.json
Đã lưu job 7 vào file job_data_1.json
✅ Đã thêm một bản ghi vào job_data_1.json
Đã lưu job 8 vào file job_data_1.json
✅ Đã thêm một bản ghi vào job_data_1.json
Đã lưu job 9 vào file job_data_1.json
✅ Đã thêm một bản ghi vào job_data_1.json
Đã lưu job 10 vào file job_data_1.json
✅ Đã thêm một bản ghi vào job_data_1.json
Đã lưu job 11 vào file job_data_1.json
❌ Lỗi khi xử lý https://www.topcv.vn/viec-lam/mobile-developer-android-developer-kotlin/1916692.html?ta_source=ITJobs_

IndexError: list index out of range